In [ ]:
#!/usr/bin/env python3
"""
Проект 7: Сравнительный анализ стратегий чанкинга для RAG

Автор: AI Assistant
Описание:
Сравнение трех стратегий разделения документов:
1. Фиксированный размер
2. Рекурсивный чанкинг
3. Семантический чанкинг
"""

import os
from typing import List

import pandas as pd
from dotenv import load_dotenv

# ------------------------------------------------------------------------------
# Загрузка переменных окружения
# ------------------------------------------------------------------------------
load_dotenv()

# ------------------------------------------------------------------------------
# ШАГ 1: ИМПОРТ БИБЛИОТЕК LANGCHAIN
# ------------------------------------------------------------------------------
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document


print("✅ Все библиотеки успешно импортированы")

# ------------------------------------------------------------------------------
# ШАГ 2: ЗАГРУЗКА ДОКУМЕНТА
# ------------------------------------------------------------------------------
def load_document(file_path: str) -> str:
    """
    Загружает текст из PDF или TXT файла.

    Args:
        file_path: путь к файлу (PDF или TXT)

    Returns:
        Текст документа
    """
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        pages = loader.load()
        text = "\n".join(page.page_content for page in pages)
        print(f"✅ PDF загружен. Всего страниц: {len(pages)}")
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        print(f"✅ Текстовый файл загружен. Размер: {len(text)} символов")

    return text


# ------------------------------------------------------------------------------
# Демонстрационный текст
# ------------------------------------------------------------------------------
document_text = load_document("document.pdf")

print(f"📄 Документ загружен. Размер: {len(document_text)} символов")
print(f"📊 Примерное количество слов: {len(document_text.split())}")

# ------------------------------------------------------------------------------
# ШАГ 3: СТРАТЕГИЯ 1 — ФИКСИРОВАННЫЙ РАЗМЕР
# ------------------------------------------------------------------------------
def strategy_1_fixed_size(
    text: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 0,
) -> List[str]:
    """
    Fixed-size chunking.
    Может разрывать предложения.
    """
    splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)


chunks_1 = strategy_1_fixed_size(document_text)

# ------------------------------------------------------------------------------
# ШАГ 4: СТРАТЕГИЯ 2 — РЕКУРСИВНЫЙ ЧАНКИНГ
# ------------------------------------------------------------------------------
def strategy_2_recursive(
    text: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
) -> List[str]:
    """
    Recursive chunking — золотой стандарт для RAG.
    """
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)


chunks_2 = strategy_2_recursive(document_text)

# ------------------------------------------------------------------------------
# ШАГ 5: СТРАТЕГИЯ 3 — СЕМАНТИЧЕСКИЙ ЧАНКИНГ
# ------------------------------------------------------------------------------
def strategy_3_semantic(text: str) -> List[str]:
    """
    Semantic chunking с использованием эмбеддингов OpenAI.
    """
    try:
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
        )
        splitter = SemanticChunker(embeddings)
        docs = splitter.create_documents([text])
        return [doc.page_content for doc in docs]

    except Exception as e:
        print(f"⚠️ SemanticChunker error: {e}")
        return []


api_key = os.getenv("OPENAI_API_KEY")
chunks_3 = strategy_3_semantic(document_text) if api_key else []

# ------------------------------------------------------------------------------
# ШАГ 6: СРАВНИТЕЛЬНЫЙ АНАЛИЗ
# ------------------------------------------------------------------------------
comparison_data = {
    "Метод": ["Фиксированный", "Рекурсивный", "Семантический"],
    "Кол-во чанков": [
        len(chunks_1),
        len(chunks_2),
        len(chunks_3) if chunks_3 else "N/A",
    ],
    "Средний размер": [
        sum(len(c) for c in chunks_1) / len(chunks_1),
        sum(len(c) for c in chunks_2) / len(chunks_2),
        sum(len(c) for c in chunks_3) / len(chunks_3) if chunks_3 else "N/A",
    ],
}

df_comparison = pd.DataFrame(comparison_data)
print("\n📊 СРАВНЕНИЕ СТРАТЕГИЙ:")
print(df_comparison)

# ------------------------------------------------------------------------------
# ШАГ 7: АНАЛИЗ СТАТИСТИКИ
# ------------------------------------------------------------------------------
def analyze_chunks(chunks: List[str]) -> dict:
    if not chunks:
        return {}

    sizes = [len(c) for c in chunks]
    avg = sum(sizes) / len(sizes)

    return {
        "total": len(chunks),
        "avg": avg,
        "min": min(sizes),
        "max": max(sizes),
        "std": (sum((s - avg) ** 2 for s in sizes) / len(sizes)) ** 0.5,
    }


print("\n📈 СТАТИСТИКА:")
print("Фиксированный:", analyze_chunks(chunks_1))
print("Рекурсивный:", analyze_chunks(chunks_2))
print("Семантический:", analyze_chunks(chunks_3))

# ------------------------------------------------------------------------------
# ШАГ 8: ВЫВОД
# ------------------------------------------------------------------------------
print(
    """
🎯 РЕКОМЕНДАЦИЯ:
Для RAG-систем оптимален Recursive Chunking:
- сохраняет структуру текста
- не требует API
- даёт лучший баланс качества и скорости

Рекомендуемые параметры:
chunk_size = 1000–1500
chunk_overlap = 150–300
"""
)

print("\n✨ ПРОЕКТ УСПЕШНО ЗАВЕРШЕН ✨")


ModuleNotFoundError: No module named 'langchain.document_loaders'